## EX04

In [4]:
import sqlite3
import pandas as pd

try:
    sqlite_connection = sqlite3.connect('../ex02/data/checking-logs.sqlite.sqlite')
except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
print('УСПЕШНО')

УСПЕШНО


In [14]:
request =  \
'''
SELECT time, avg(diff)
FROM (SELECT uid,
             CAST( (JulianDay(test.first_commit_ts) - 
                    JulianDay(datetime(deadlines.deadlines, 'unixepoch'))
                    ) * 24 as Integer
                  ) as diff,
             CASE
                  WHEN
                        test.first_commit_ts < test.first_view_ts THEN 'before'
                  ELSE
                        'after' END
            AS time

       FROM test left join deadlines on test.labname = deadlines.labs
       WHERE labname != 'project1'
      )

WHERE uid in (SELECT uid
              FROM (SELECT uid,
                  CASE
                        WHEN
                              test.first_commit_ts < test.first_view_ts THEN 'before'
                        ELSE 
                              'after' END 
                        AS time
                    FROM test
                    LEFT JOIN deadlines ON test.labname=deadlines.labs
                    WHERE labname != 'project1'
                    )
               GROUP BY uid
               HAVING COUNT(DISTINCT time)=2
               )
GROUP BY time
'''
test_results = pd.io.sql.read_sql(request, sqlite_connection)
test_results

,time,avg(diff)
0,after,-104.6000
1,before,-60.5625


In [18]:
request = \
'''
SELECT time, avg(diff)
FROM (SELECT uid,
             CAST( (JulianDay(control.first_commit_ts) -
                    JulianDay(datetime(deadlines.deadlines, 'unixepoch'))
                    ) * 24 as Integer
                  ) as diff,
             CASE 
             WHEN control.first_commit_ts < control.first_view_ts 
                  THEN 'before'
             ELSE 
                  'after' END 
            AS time
            FROM control
            LEFT JOIN deadlines ON control.labname=deadlines.labs
            WHERE labname != 'project1'
      )
WHERE uid IN (SELECT uid
              FROM (SELECT uid,
                           CASE
                              WHEN control.first_commit_ts < control.first_view_ts
                                     THEN 'before'
                              ELSE 
                                    'after' END 
                              AS time
                    FROM control
                    LEFT JOIN deadlines ON control.labname=deadlines.labs
                    WHERE labname != 'project1'
                    )
              GROUP BY uid
              HAVING COUNT(distinct time)=2
              )
GROUP BY time
'''
control_results = pd.io.sql.read_sql(request, sqlite_connection)
control_results

,time,avg(diff)
0,after,-117.636364
1,before,-99.464286


In [ ]:
sqlite_connection.close()

## have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?
YES